In [2]:
# Load weather data
import pandas as pd

data = pd.read_csv("../data/combined_DFW_weather.csv")
data.head()

/var/folders/5v/2ljsc8_n2cv62zbxy_jkx52h0000gn/T/ipykernel_3527/419576524.py:4: DtypeWarning: Columns (22,23,24,25,26,27,36,43,44,45,56,57,58,62,91,100,103,104,105) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/combined_DFW_weather.csv")


,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,...,OE3,RH1,RH2,RH3,WA1,REM,EQD,AU4,AT8,AW5
0,72259003927,2021-01-01T00:00:00,4,32.89744,-97.02196,165.7,"DAL FTW WSCMO AIRPORT, TX US",FM-12,99999,V020,...,NaN,NaN,NaN,NaN,NaN,SYN09872259 11366 80318 10033 20028 39821 4002...,NaN,NaN,NaN,NaN
1,72259003927,2021-01-01T00:53:00,7,32.89744,-97.02196,165.7,"DAL FTW WSCMO AIRPORT, TX US",FM-15,KDFW,V030,...,NaN,NaN,NaN,NaN,NaN,MET12012/31/20 18:53:03 METAR KDFW 010053Z 020...,NaN,NaN,NaN,NaN
2,72259003927,2021-01-01T01:24:00,7,32.89744,-97.02196,165.7,"DAL FTW WSCMO AIRPORT, TX US",FM-16,KDFW,V030,...,NaN,NaN,NaN,NaN,NaN,MET19112/31/20 19:24:03 SPECI KDFW 010124Z 020...,NaN,NaN,NaN,NaN
3,72259003927,2021-01-01T01:51:00,6,32.89744,-97.02196,165.7,"DAL FTW WSCMO AIRPORT, TX US",FM-16,KDFW,V030,...,NaN,NaN,NaN,NaN,NaN,MET19412/31/20 19:51:03 SPECI KDFW 010151Z 360...,NaN,NaN,NaN,NaN
4,72259003927,2021-01-01T01:53:00,7,32.89744,-97.02196,165.7,"DAL FTW WSCMO AIRPORT, TX US",FM-15,KDFW,V030,...,NaN,NaN,NaN,NaN,NaN,MET20212/31/20 19:53:03 METAR KDFW 010153Z 350...,NaN,NaN,NaN,NaN


In [8]:
data2 = data[['STATION', 'DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION',
       'NAME', 'REPORT_TYPE', 'CALL_SIGN', 'QUALITY_CONTROL', 'WND', 'CIG',
       'VIS', 'TMP', 'DEW', 'SLP']].copy()

In [9]:
data2.columns

Index(['STATION', 'DATE', 'SOURCE', 'LATITUDE', 'LONGITUDE', 'ELEVATION',
       'NAME', 'REPORT_TYPE', 'CALL_SIGN', 'QUALITY_CONTROL', 'WND', 'CIG',
       'VIS', 'TMP', 'DEW', 'SLP'],
      dtype='object')

In [12]:
data2['TMP']

0        +0033,1
1        +0033,5
2        +0033,5
3        +0030,5
4        +0033,5
          ...   
53849    +0139,1
53850    +0139,1
53851    +0139,1
53852    +0128,1
53853    +0106,1
Name: TMP, Length: 53854, dtype: object

In [14]:
data2["TEMP_val"] = data2["TMP"].str.split(",", expand=True)[0].astype(int) / 10

In [17]:
import pandas as pd

# 1. Load your combined DFW file
df = pd.read_csv("../data/combined_DFW_weather.csv")

# ---------------------------
#  Helper: split "value,QC"
# ---------------------------
def split_grouped(series):
    s = series.astype(str).str.split(",", expand=True)
    s.columns = ["value", "qc"]
    s["value"] = pd.to_numeric(s["value"], errors="coerce")
    s["qc"] = pd.to_numeric(s["qc"], errors="coerce")
    return s

# ---------------------------------
# 2. Temperature (°C) and Dew Point
# ---------------------------------
if "TEMP" in df.columns:
    temp = split_grouped(df["TEMP"])
    df["TEMP_C"] = temp["value"].replace(9999, pd.NA) / 10
    df["TEMP_qc"] = temp["qc"]

if "DEW" in df.columns:
    dew = split_grouped(df["DEW"])
    df["DEW_C"] = dew["value"].replace(9999, pd.NA) / 10
    df["DEW_qc"] = dew["qc"]

# --------------------------------
# 3. Pressures (hPa) — SLP & STP
# --------------------------------
if "SLP" in df.columns:
    slp = split_grouped(df["SLP"])
    df["SLP_hPa"] = slp["value"].replace(99999, pd.NA) / 10
    df["SLP_qc"]  = slp["qc"]

if "STP" in df.columns:
    stp = split_grouped(df["STP"])
    df["STP_hPa"] = stp["value"].replace(99999, pd.NA) / 10
    df["STP_qc"]  = stp["qc"]

# ---------------------------
# 4. Visibility (meters)
# ---------------------------
if "VIS" in df.columns:
    vis = split_grouped(df["VIS"])
    df["VIS_m"] = vis["value"].replace(999999, pd.NA)
    df["VIS_qc"] = vis["qc"]

# -------------------------------------
# 5. Wind — direction + speed (m/s)
# -------------------------------------
if "WND" in df.columns:
    wnd = split_grouped(df["WND"])
    
    v = wnd["value"]
    df["WIND_DIR_deg"] = (v // 1000).replace(999, pd.NA)
    df["WIND_SPD_ms"]  = ((v % 1000) / 10).replace(9999/10, pd.NA)
    df["WIND_qc"]      = wnd["qc"]

# ------------------------------
# 6. Wind Gust (m/s)
# ------------------------------
if "GUST" in df.columns:
    gust = split_grouped(df["GUST"])
    df["GUST_ms"] = gust["value"].replace(9999, pd.NA) / 10
    df["GUST_qc"] = gust["qc"]

# ----------------------------------------------------
# 7. Convert DATE + TIME columns → full datetime
# ----------------------------------------------------
# NOAA ISD uses: DATE = YYYYMMDD, TIME = HHMM
df["DATE"] = df["DATE"].astype(str)
df["TIME"] = df["TIME"].apply(lambda x: f"{int(x):04d}")  # zero-pad

df["datetime"] = pd.to_datetime(df["DATE"] + df["TIME"], format="%Y%m%d%H%M")

# ----------------------------------------------------
# 8. Sort and show final cleaned dataset
# ----------------------------------------------------
df = df.sort_values("datetime").reset_index(drop=True)

df[["datetime", "TEMP_C", "DEW_C", "WIND_DIR_deg", "WIND_SPD_ms", 
    "SLP_hPa", "STP_hPa", "VIS_m", "GUST_ms"]].head()

/var/folders/5v/2ljsc8_n2cv62zbxy_jkx52h0000gn/T/ipykernel_3527/1193560680.py:4: DtypeWarning: Columns (22,23,24,25,26,27,36,43,44,45,56,57,58,62,91,100,103,104,105) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/combined_DFW_weather.csv")


ValueError: Length mismatch: Expected axis has 4 elements, new values have 2 elements